# 텐서플로2와 머신러닝으로 ~~~



## P74 - konlpy

In [66]:
from konlpy.tag import Okt
from hanspell import spell_checker

okt = Okt()

text = "룰루랄라 신나는 하루가 되길 바래는 아침"

result = spell_checker.check(text).checked

print(okt.morphs(text))
print(okt.morphs(text, stem=True))

#print(result)
print(okt.morphs(result, stem=True))

['룰루랄라', '신나는', '하루', '가', '되길', '바', '래', '는', '아침']
['룰루랄라', '신나다', '하루', '가', '되다', '바', '래', '늘다', '아침']
['룰루랄라', '신나다', '하루', '가', '되다', '바라다', '아침']
